In [1]:
using Revise
using QuantumOptimalControl
using QuantumOptics
using Flux, DiffEqFlux
using PlotlyJS
using DifferentialEquations: DP5, Tsit5, Vern7, Vern9, BS3
using Random
using ProgressMeter
ProgressMeter.ijulia_behavior(:clear)

WebIO._IJuliaInit()

┌ Info: Precompiling QuantumOptimalControl [e91afccf-c93e-44fd-aa4c-3d3ef13645c0]
└ @ Base loading.jl:1423


false

In [2]:
bs = SpinBasis(1//2)
n_neurons = 8
Random.seed!(1)
ann = FastChain(FastDense(1, n_neurons, tanh), 
                FastDense(n_neurons, n_neurons, tanh),
                FastDense(n_neurons, n_neurons, tanh),
                FastDense(n_neurons, 1))
θ = Vector{Float64}(initial_params(ann));
n_params = length(θ)
println(n_params)
(t0, t1)=(0.0, 5.0)

169


(0.0, 5.0)

In [3]:
bcs(params) = ((ann([t0], params)[1])^2 + (ann([t1], params)[1])^2)
cost = CostFunction((x,y)->sum(abs.(x-y).^2), bcs)

CostFunction(var"#1#2"(), bcs)

In [4]:
coeffs(params, t) = ann([t], params)[1]
tspan = (t0, t1)
trans = vectorize(StateTransform(spindown(bs)=>spinup(bs)))
J = [sigmam(bs)]
rates = [0.001]
H = Hamiltonian(-0.5*sigmaz(bs), [sigmax(bs)], coeffs)
Hv = vectorize(H, J, rates);

In [5]:
prob = QOCProblem(Hv, trans, tspan, cost);

In [6]:
@time sol = solve(prob, θ, ADAM(0.01); maxiter=200, abstol=1e-6, reltol=1e-6)

Progress: 100%|█████████████████████████████████████████| Time: 0:01:38
  distance:     0.00037648419662200157
  constraints:  0.00015189241911569857


128.244583 seconds (213.84 M allocations: 15.246 GiB, 53.45% gc time, 43.84% compilation time)


Solution{Float64}([-0.8993780362489809, -0.5628423663087397, -0.2551807906783756, -0.6670881639664165, -1.2665580855810907, 0.4667991965760212, -0.6883618053692558, 0.41646966239290134, 0.3871304745037402, 0.37066721083520493  …  0.014521679176182144, 0.9027965293915926, -0.2861745522619893, -0.3345035526168799, 0.9569130150889426, 0.8077440944394668, 0.708285507230912, -0.3581830009142783, 0.6094401556399135, -0.02730977997742402], [1.8857371931041378, 1.982260559547704, 1.9717257561376687, 1.9348768891795805, 1.9191418233662174, 1.895746691649922, 1.858531011090265, 1.7874310915040985, 1.6286803465150375, 1.3638261189502698  …  0.0011282618730790945, 0.00011813107084023737, 0.0007294954574232115, 0.0005176354040146317, 0.0002822110854102594, 0.00042677449755188954, 0.00013385253588962077, 0.0005726546934797629, 8.252979844012601e-5, 0.00037648419662200157], [0.12094647837122652, 0.011290829951537311, 0.02374810335493969, 0.05055197513465032, 0.0631159047715699, 0.04477394243990461, 0

In [7]:
plot(sol.distance_trace)

data: [
  "scatter with fields type, x, and y"
]

layout: "layout with fields margin and template"

In [8]:
ts = t0:t1/100:t1
Ω(t) = coeffs(sol.params, t)[1]
f = plot([scatter(x=ts, y=Ω.(ts), name="Ω")],
     Layout(
        xaxis_title_text="Time (a.u)",
        yaxis_title_text="Ω (a.u)",
        legend=attr(x=0, y=1,),
        font=attr(
            size=16
        )
    )
)

data: [
  "scatter with fields name, type, x, and y"
]

layout: "layout with fields font, legend, margin, template, xaxis, and yaxis"

In [9]:
tout, psit = master_dynamic(ts, spindown(bs), H, sol.params, J, rates);

In [10]:
f = plot(
    [
        scatter(x=tout, y=real(expect(dm(spindown(bs)), psit)), name="|↓⟩")
        scatter(x=tout, y=real(expect(dm(spinup(bs)), psit)), name="|↑⟩")
    ],
    Layout(
        xaxis_title_text="Time (a.u)",
        yaxis_title_text="Population (⟨i|ψ⟩²)",
        legend=attr(x=0.75, y=0.5),
        font=attr(
            size=16
        )
     )
)

data: [
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y"
]

layout: "layout with fields font, legend, margin, template, xaxis, and yaxis"